In [1]:
import pandas as pd

df = pd.read_csv("AccountLevelCleaned.csv")
game_df = pd.read_csv("../data/Prompt1GameLevel.csv") 
seat_df = pd.read_csv("../data/Prompt1SeatLevel.csv")

df.drop(columns=["SumGamesAttended", "TotNumTicketsPurchased"], inplace=True)

df.head()

,Season,AccountNumber,SingleGameTickets,PartialPlanTickets,GroupTickets,AvgSpend,GamesAttended,FanSegment,DistanceToArena,BasketballPropensity,SocialMediaEngagement
0,2023,1,0,0,0,467.0,0,F,12.0,872.0,Low
1,2023,2,2,0,0,116.0,1,A,47.0,485.0,Low
2,2023,3,3,0,0,107.0,1,B,6.0,896.0,Low
3,2023,4,0,0,3,27.0,1,C,3.0,467.0,High
4,2023,5,0,0,2,14.0,1,A,4.0,582.0,Medium


In [2]:
# Compute the number of unique games attended per account per season
unique_games_count = seat_df.groupby(["Season", "AccountNumber"])["Game"].nunique().reset_index()
unique_games_count.rename(columns={"Game": "UniqueGamesAttended"}, inplace=True)

# Compute the total number of tickets purchased per account per season
total_tickets_count = seat_df.groupby(["Season", "AccountNumber"]).size().reset_index(name="TotalTicketsPurchased")

# Merge the computed values into account_level_df
df = df.merge(unique_games_count, on=["Season", "AccountNumber"], how="left")
df = df.merge(total_tickets_count, on=["Season", "AccountNumber"], how="left")

# Fill NaN values with 0 (in case an account didn't attend any games)
df.fillna(0, inplace=True)

# Display the updated dataset
df.head()

,Season,AccountNumber,SingleGameTickets,PartialPlanTickets,GroupTickets,AvgSpend,GamesAttended,FanSegment,DistanceToArena,BasketballPropensity,SocialMediaEngagement,UniqueGamesAttended,TotalTicketsPurchased
0,2023,1,0,0,0,467.0,0,F,12.0,872.0,Low,1,24
1,2023,2,2,0,0,116.0,1,A,47.0,485.0,Low,1,2
2,2023,3,3,0,0,107.0,1,B,6.0,896.0,Low,1,3
3,2023,4,0,0,3,27.0,1,C,3.0,467.0,High,1,3
4,2023,5,0,0,2,14.0,1,A,4.0,582.0,Medium,1,2


In [3]:
df["AvgSpend"].describe()



count    42016.000000
mean        76.330044
std         82.422255
min          0.000000
25%         29.000000
50%         60.000000
75%         96.000000
max       3297.000000
Name: AvgSpend, dtype: float64

In [4]:
# Identify outliers using the IQR method
Q1 = df["AvgSpend"].quantile(0.25)
Q3 = df["AvgSpend"].quantile(0.75)
IQR = Q3 - Q1

# Define outlier boundaries
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Cap the outliers to the upper bound
df["AvgSpend_capped"] = df["AvgSpend"].clip(lower=lower_bound, upper=upper_bound)

# Normalize the capped AvgSpend values using Min-Max Scaling
max_avgspend_capped = df["AvgSpend_capped"].max()
min_avgspend_capped = df["AvgSpend_capped"].min()

df["AvgSpend_norm"] = (df["AvgSpend_capped"] - min_avgspend_capped) / (max_avgspend_capped - min_avgspend_capped)

# Show summary statistics after outlier handling and normalization
df[["AvgSpend", "AvgSpend_capped", "AvgSpend_norm"]].describe()


,AvgSpend,AvgSpend_capped,AvgSpend_norm
count,42016.000000,42016.000000,42016.000000
mean,76.330044,69.974953,0.356107
std,82.422255,52.404021,0.266687
min,0.000000,0.000000,0.000000
25%,29.000000,29.000000,0.147583
50%,60.000000,60.000000,0.305344
75%,96.000000,96.000000,0.488550
max,3297.000000,196.500000,1.000000


In [5]:
df['DistanceToArena'].describe()


count    42016.000000
mean       148.744570
std        321.795226
min          0.000000
25%          9.000000
50%         45.000000
75%        119.000000
max       4240.000000
Name: DistanceToArena, dtype: float64

In [6]:
# Observe DistanceToArena statistics and check for outliers
distance_stats = df["DistanceToArena"].describe()

# Identify outliers using the IQR method
Q1_distance = df["DistanceToArena"].quantile(0.25)
Q3_distance = df["DistanceToArena"].quantile(0.75)
IQR_distance = Q3_distance - Q1_distance

# Define outlier boundaries
lower_bound_distance = Q1_distance - 1.5 * IQR_distance
upper_bound_distance = Q3_distance + 1.5 * IQR_distance

# Cap the outliers to the upper bound
df["DistanceToArena_capped"] = df["DistanceToArena"].clip(lower=lower_bound_distance, upper=upper_bound_distance)

# Normalize using Min-Max Scaling after capping
max_distance_capped = df["DistanceToArena_capped"].max()
min_distance_capped = df["DistanceToArena_capped"].min()

df["DistanceToArena_norm"] = (df["DistanceToArena_capped"] - min_distance_capped) / (max_distance_capped - min_distance_capped)

# Show summary statistics after outlier handling and normalization
df[["DistanceToArena", "DistanceToArena_capped", "DistanceToArena_norm"]].describe()

,DistanceToArena,DistanceToArena_capped,DistanceToArena_norm
count,42016.000000,42016.000000,42016.000000
mean,148.744570,79.915172,0.281391
std,321.795226,90.356100,0.318155
min,0.000000,0.000000,0.000000
25%,9.000000,9.000000,0.031690
50%,45.000000,45.000000,0.158451
75%,119.000000,119.000000,0.419014
max,4240.000000,284.000000,1.000000


In [7]:
# Compute the 95th percentile threshold for TotNumTicketsPurchased
percentile_95_tickets = df["TotalTicketsPurchased"].quantile(0.95)

# Cap extreme values at the 99th percentile
df["TotalTicketsPurchased_capped"] = df["TotalTicketsPurchased"].clip(upper=percentile_95_tickets)

# Normalize again using Min-Max Scaling after capping
max_tickets_capped = df["TotalTicketsPurchased_capped"].max()
min_tickets_capped = df["TotalTicketsPurchased_capped"].min()

df["TotalTicketsPurchased_norm"] = (df["TotalTicketsPurchased_capped"] - min_tickets_capped) / (max_tickets_capped - min_tickets_capped)

# Show summary statistics after percentile-based capping and normalization
df[["TotalTicketsPurchased", "TotalTicketsPurchased_capped", "TotalTicketsPurchased_norm"]].describe()

,TotalTicketsPurchased,TotalTicketsPurchased_capped,TotalTicketsPurchased_norm
count,42016.000000,42016.000000,42016.000000
mean,5.312809,3.810763,0.187384
std,31.085796,3.502422,0.233495
min,1.000000,1.000000,0.000000
25%,2.000000,2.000000,0.066667
50%,3.000000,3.000000,0.133333
75%,4.000000,4.000000,0.200000
max,4094.000000,16.000000,1.000000


In [8]:
# Compute the 99th percentile thresholds for each games attended column
percentile_99_games_attended = df["GamesAttended"].quantile(0.99)
percentile_99_sum_games_attended = df["UniqueGamesAttended"].quantile(0.99)

# Cap extreme values at the 99th percentile for each column
df["GamesAttended_capped"] = df["GamesAttended"].clip(upper=percentile_99_games_attended)
df["UniqueGamesAttended_capped"] = df["UniqueGamesAttended"].clip(upper=percentile_99_sum_games_attended)

# Normalize using Min-Max Scaling after winsorization
max_games_attended_capped = df["GamesAttended_capped"].max()
min_games_attended_capped = df["GamesAttended_capped"].min()
df["GamesAttended_norm"] = (df["GamesAttended_capped"] - min_games_attended_capped) / (max_games_attended_capped - min_games_attended_capped)

max_sum_games_attended_capped = df["UniqueGamesAttended_capped"].max()
min_sum_games_attended_capped = df["UniqueGamesAttended_capped"].min()
df["UniqueGamesAttended_norm"] = (df["UniqueGamesAttended_capped"] - min_sum_games_attended_capped) / (max_sum_games_attended_capped - min_sum_games_attended_capped)

# Show summary statistics after percentile-based capping and normalization
df[["GamesAttended", "GamesAttended_capped", "GamesAttended_norm", 
    "UniqueGamesAttended", "UniqueGamesAttended_capped", "UniqueGamesAttended_norm"]].describe()

,GamesAttended,GamesAttended_capped,GamesAttended_norm,UniqueGamesAttended,UniqueGamesAttended_capped,UniqueGamesAttended_norm
count,42016.000000,42016.000000,42016.000000,42016.000000,42016.000000,42016.000000
mean,0.972796,0.933264,0.186653,1.552075,1.466299,0.033307
std,1.170545,0.770960,0.154192,2.752490,1.980196,0.141443
min,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,1.000000,1.000000,0.200000,1.000000,1.000000,0.000000
50%,1.000000,1.000000,0.200000,1.000000,1.000000,0.000000
75%,1.000000,1.000000,0.200000,1.000000,1.000000,0.000000
max,38.000000,5.000000,1.000000,41.000000,15.000000,1.000000


In [9]:
# Normalize BasketballPropensity by dividing by 1000
df["BasketballPropensity_norm"] = df["BasketballPropensity"] / 1000

# Show summary statistics after normalization
df[["BasketballPropensity", "BasketballPropensity_norm"]].describe()

,BasketballPropensity,BasketballPropensity_norm
count,42016.000000,42016.000000
mean,683.505458,0.683505
std,216.242506,0.216243
min,125.000000,0.125000
25%,507.000000,0.507000
50%,683.505458,0.683505
75%,908.000000,0.908000
max,993.000000,0.993000


In [10]:
# Map SocialMediaEngagement categories to numerical values
social_media_mapping = {
    "Low": 0.3,
    "Medium": 0.6,
    "High": 1.0
}

df["SocialMediaEngagement_norm"] = df["SocialMediaEngagement"].map(social_media_mapping)

# Show summary statistics after mapping
df[["SocialMediaEngagement", "SocialMediaEngagement_norm"]].describe()

,SocialMediaEngagement_norm
count,42016.000000
mean,0.625257
std,0.248689
min,0.300000
25%,0.600000
50%,0.600000
75%,1.000000
max,1.000000


In [11]:
# Count the number of games per tier for each AccountNumber and Season


# Count the number of games per tier for each AccountNumber and Season
tier_counts = seat_df.groupby(["Season", "AccountNumber", "GameTier"]).size().unstack(fill_value=0)

# Rename columns to indicate they represent tier counts
tier_counts.columns = [f"Tier_{col}_Count" for col in tier_counts.columns]
                                                                                      
# Reset index to keep AccountNumber and Season as columns
tier_counts.reset_index(inplace=True)

df = df.merge(tier_counts, on=["Season", "AccountNumber"], how="left")
df.fillna(0, inplace=True)

df.head()


,Season,AccountNumber,SingleGameTickets,PartialPlanTickets,GroupTickets,AvgSpend,GamesAttended,FanSegment,DistanceToArena,BasketballPropensity,...,GamesAttended_capped,UniqueGamesAttended_capped,GamesAttended_norm,UniqueGamesAttended_norm,BasketballPropensity_norm,SocialMediaEngagement_norm,Tier_A_Count,Tier_B_Count,Tier_C_Count,Tier_D_Count
0,2023,1,0,0,0,467.0,0,F,12.0,872.0,...,0,1,0.0,0.0,0.872,0.3,0,0,0,24
1,2023,2,2,0,0,116.0,1,A,47.0,485.0,...,1,1,0.2,0.0,0.485,0.3,2,0,0,0
2,2023,3,3,0,0,107.0,1,B,6.0,896.0,...,1,1,0.2,0.0,0.896,0.3,0,0,0,3
3,2023,4,0,0,3,27.0,1,C,3.0,467.0,...,1,1,0.2,0.0,0.467,1.0,0,0,0,3
4,2023,5,0,0,2,14.0,1,A,4.0,582.0,...,1,1,0.2,0.0,0.582,0.6,0,0,0,2


In [12]:
# Compute fractions for each tier
df["Tier_A_Fraction"] = df["Tier_A_Count"] / df["TotalTicketsPurchased"]
df["Tier_B_Fraction"] = df["Tier_B_Count"] / df["TotalTicketsPurchased"]
df["Tier_C_Fraction"] = df["Tier_C_Count"] / df["TotalTicketsPurchased"]
df["Tier_D_Fraction"] = df["Tier_D_Count"] / df["TotalTicketsPurchased"]

# Compute combined fractions for Tier C + D and Tier A + B
df["Tier_CD_Fraction"] = (df["Tier_C_Count"] + df["Tier_D_Count"]) / df["TotalTicketsPurchased"]
df["Tier_AB_Fraction"] = (df["Tier_A_Count"] + df["Tier_B_Count"]) / df["TotalTicketsPurchased"]

# Fill NaN values (which occur if TotalTicketsPurchased was 0) with 0
df.fillna(0, inplace=True)

# Determine the highest attended tier per account
df["MostAttendedTier"] = df[["Tier_A_Count", "Tier_B_Count", "Tier_C_Count", "Tier_D_Count"]].idxmax(axis=1)
df["MostAttendedTier"] = df["MostAttendedTier"].str.replace("_Count", "")

df[["Tier_A_Fraction", "Tier_B_Fraction", "Tier_C_Fraction", "Tier_D_Fraction", "Tier_CD_Fraction", "Tier_AB_Fraction"]].describe()


,Tier_A_Fraction,Tier_B_Fraction,Tier_C_Fraction,Tier_D_Fraction,Tier_CD_Fraction,Tier_AB_Fraction
count,42016.000000,42016.000000,42016.000000,42016.000000,42016.000000,42016.000000
mean,0.112346,0.160463,0.304647,0.422545,0.727192,0.272808
std,0.302976,0.351819,0.443896,0.476608,0.427794,0.427794
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000
50%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,0.000000,0.000000,1.000000,1.000000,1.000000,0.666667
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
seat_df["GameDate"] = pd.to_datetime(seat_df["GameDate"])

# Identify whether each game was on a weekend (Saturday or Sunday)
seat_df["IsWeekend"] = seat_df["GameDate"].dt.dayofweek.isin([4, 5, 6]).astype(int)

seat_df.head()

,Season,AccountNumber,Game,GameDate,GameTier,IsWeekend
0,2023,1,2024-01-24 Cleveland Cavaliers,2024-01-24,D,0
1,2023,1,2024-01-24 Cleveland Cavaliers,2024-01-24,D,0
2,2023,1,2024-01-24 Cleveland Cavaliers,2024-01-24,D,0
3,2023,1,2024-01-24 Cleveland Cavaliers,2024-01-24,D,0
4,2023,1,2024-01-24 Cleveland Cavaliers,2024-01-24,D,0


In [14]:
# Compute fraction of weekend games attended per account
weekend_ratio = seat_df.groupby(["Season", "AccountNumber"])["IsWeekend"].mean().reset_index()
weekend_ratio.rename(columns={"IsWeekend": "WeekendFraction"}, inplace=True)

df = df.merge(weekend_ratio, on=["Season", "AccountNumber"], how="left")
# Fill NaN values with 0 (for accounts with no game attendance)
df.fillna(0, inplace=True)

df["WeekendFraction"].describe()


count    42016.000000
mean         0.500877
std          0.482775
min          0.000000
25%          0.000000
50%          0.500000
75%          1.000000
max          1.000000
Name: WeekendFraction, dtype: float64

In [15]:
# Identify whether each game was on a weekday (Monday-Friday)
seat_df["IsWeekday"] = (seat_df["GameDate"].dt.dayofweek < 4).astype(int)

# Compute weekday fractions for Tier A/B and Tier C/D per account
tier_weekday_counts = seat_df.groupby(["Season", "AccountNumber", "GameTier"])["IsWeekday"].mean().unstack(fill_value=0)

# Rename columns
tier_weekday_counts = tier_weekday_counts.rename(columns={
    "A": "TierA_Weekday_Fraction",
    "B": "TierB_Weekday_Fraction",
    "C": "TierC_Weekday_Fraction",
    "D": "TierD_Weekday_Fraction"
}).reset_index()

# Compute combined TierAB and TierCD weekday fractions
tier_weekday_counts["TierAB_Weekday_Fraction"] = tier_weekday_counts["TierA_Weekday_Fraction"] + tier_weekday_counts["TierB_Weekday_Fraction"]
tier_weekday_counts["TierCD_Weekday_Fraction"] = tier_weekday_counts["TierC_Weekday_Fraction"] + tier_weekday_counts["TierD_Weekday_Fraction"]

# Merge weekday fractions into account-level dataset
df = df.merge(tier_weekday_counts, on=["Season", "AccountNumber"], how="left")

# Fill NaN values with 0 (for accounts with no weekday game attendance)
df.fillna(0, inplace=True)


In [16]:
# Mark games that had a giveaway (non-empty giveaway field)
game_df["HasGiveaway"] = game_df["Giveaway"].notna().astype(int)

# Merge giveaway information into the seat-level dataset
seat_df = seat_df.merge(game_df[["Game", "HasGiveaway"]], on="Game", how="left")

# Compute fraction of attended games that were giveaway games per account
giveaway_fraction = seat_df.groupby(["Season", "AccountNumber"])["HasGiveaway"].mean().reset_index()
giveaway_fraction.rename(columns={"HasGiveaway": "GiveawayFraction"}, inplace=True)

# Merge giveaway fraction into account-level dataset
df = df.merge(giveaway_fraction, on=["Season", "AccountNumber"], how="left")

# Fill NaN values with 0 (for accounts that did not attend any giveaway games)
df.fillna(0, inplace=True)

# Save the updated account-level file
updated_account_file_path = "AccountLevelFinal.csv"
df.to_csv(updated_account_file_path, index=False)

# Provide the updated file to the user
updated_account_file_path


'AccountLevelFinal.csv'